## Docking with Smina 

This Google Colab notebook is inspired from [the post of **Esben Jannik Bjerrum** about using smina for docking](https://www.wildcardconsulting.dk/ligand-docking-with-smina/)

## Step 1. Install OpenBabel, RDKit and Py3Dmol

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit
!time conda install -q -y -c openbabel openbabel
!pip install py3Dmol # 3D Molecular Visualizer
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-04-21 01:45:23--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2021-04-21 01:45:23--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94235922 (90M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  89.87M   180MB/s    in 0.5s    

2021-04-21 01:45:24 (180 MB/s) - ‘Miniconda3-latest-Linux

In [ ]:
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import DataStructs
from rdkit.Chem import RDConfig
from rdkit.Chem import rdBase
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(rdBase.rdkitVersion)

2019.03.4


In [ ]:
#!pip install mdtraj

     |████████████████████████████████| 911kB 2.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 25.2MB 33.7MB/s 
  Created wheel for mdtraj: filename=mdtraj-1.9.3-cp37-cp37m-linux_x86_64.whl size=5727760 sha256=d90640980faff023b123efa78b2052c5a20a6878569c5d809e4c1e52967e1ea0
  Stored in directory: /root/.cache/pip/wheels/7d/b5/5b/a11d3ec14d2921175986eacba4e5fb9d87fb545b4586de3dfd
Successfully built mdtraj


In [ ]:
#import mdtraj as md

## Step2: Download Smina and Give Necessary Permissions

In [ ]:
!wget https://sourceforge.net/projects/smina/files/smina.static/download -O smina.static

--2019-11-03 14:02:47--  https://sourceforge.net/projects/smina/files/smina.static/download
Resolving sourceforge.net (sourceforge.net)... 216.105.38.13
Connecting to sourceforge.net (sourceforge.net)|216.105.38.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.sourceforge.net/project/smina/smina.static?r=&ts=1572789767&use_mirror=iweb [following]
--2019-11-03 14:02:47--  https://downloads.sourceforge.net/project/smina/smina.static?r=&ts=1572789767&use_mirror=iweb
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 216.105.38.13
Connecting to downloads.sourceforge.net (downloads.sourceforge.net)|216.105.38.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://iweb.dl.sourceforge.net/project/smina/smina.static [following]
--2019-11-03 14:02:48--  https://iweb.dl.sourceforge.net/project/smina/smina.static
Resolving iweb.dl.sourceforge.net (iweb.dl.sourceforge.net)... 192.175.120.182, 2607

In [ ]:
!chmod u+x smina.static

In [ ]:
!./smina.static

Missing receptor.

Correct usage:

Input:
  -r [ --receptor ] arg         rigid part of the receptor (PDBQT)
  --flex arg                    flexible side chains, if any (PDBQT)
  -l [ --ligand ] arg           ligand(s)
  --flexres arg                 flexible side chains specified by comma 
                                separated list of chain:resid or 
                                chain:resid:icode
  --flexdist_ligand arg         Ligand to use for flexdist
  --flexdist arg                set all side chains within specified distance 
                                to flexdist_ligand to flexible

Search space (required):
  --center_x arg                X coordinate of the center
  --center_y arg                Y coordinate of the center
  --center_z arg                Z coordinate of the center
  --size_x arg                  size in the X dimension (Angstroms)
  --size_y arg                  size in the Y dimension (Angstroms)
  --size_z arg                  size in the Z dimen

## Step3: Install Pymol for PDB manipulations

In [ ]:
! apt-get install pymol 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  apbs libglew2.0 libmaloc1 pymol-data python-opengl python-pmw
Suggested packages:
  glew-utils libgle3 python-pmw-doc
The following NEW packages will be installed:
  apbs libglew2.0 libmaloc1 pymol pymol-data python-opengl python-pmw
0 upgraded, 7 newly installed, 0 to remove and 28 not upgraded.
Need to get 5,912 kB of archives.
After this operation, 26.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmaloc1 amd64 0.2-3.1 [48.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 apbs amd64 1.4-1build1 [218 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libglew2.0 amd64 2.0.0-5 [140 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-pmw all 1.3.2-6build1 [79.5 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 pyth

### Creates Pymol script necessary for seperating Protein and Native Ligand

In [ ]:
#!wget -O 1OYT.pdb https://files.rcsb.org/download/1oyt.pdb
com_file = open('fetch_and_clean.pml','w')
com_file.write('''
load 1OYT.pdb
remove resn HOH
h_add elem O or elem N
select 1OYT-FSN, resn FSN #Create a selection called 1OYT-FSN from the ligand
select 1OYT-receptor, 1OYT and not 1OYT-FSN #Select all that is not the ligand
save 1OYT-FSN.pdb, 1OYT-FSN
save 1OYT-receptor.pdb, 1OYT-receptor    
''')
com_file.close()

In [ ]:
!pymol -c fetch_and_clean.pml

 PyMOL(TM) Molecular Graphics System, Version 1.8.4.0.
 Copyright (c) Schrodinger, LLC.
 All Rights Reserved.
 
    Created by Warren L. DeLano, Ph.D. 
 
    PyMOL is user-supported open-source software.  Although some versions
    are freely available, PyMOL is not in the public domain.
 
    If PyMOL is helpful in your work or study, then please volunteer 
    support for our ongoing efforts to create open and affordable scientific
    software by purchasing a PyMOL Maintenance and/or Support subscription.

    More information can be found at "http://www.pymol.org".
 
    Enter "help" for a list of commands.
    Enter "help <command-name>" for information on a specific command.

 Hit ESC anytime to toggle between text and graphics.

 Command mode. No graphics front end.
 Detected 2 CPU cores.  Enabled multithreaded rendering.
PyMOL>load 1OYT.pdb
HEADER    HYDROLASE/HYDROLASE INHIBITOR           07-APR-03   1OYT
TITLE     COMPLEX OF RECOMBINANT HUMAN THROMBIN WITH A DESIGNED FLUORINA

### Convert receptor and native ligands to PDBQT format with OpenBabel

In [ ]:
!obabel 1OYT-receptor.pdb -xr -O 1OYT-receptor.pdbqt
!obabel 1OYT-FSN.pdb -O 1OYT-FSN.pdbqt

I: identified residue wrt insertion code: 'A'
I: identified residue wrt insertion code: 'A'
I: identified residue wrt insertion code: 'A'
I: identified residue wrt insertion code: 'A'
I: identified residue wrt insertion code: 'A'
I: identified residue wrt insertion code: 'A'
I: identified residue wrt insertion code: 'A'
I: identified residue wrt insertion code: 'F'
I: identified residue wrt insertion code: 'F'
I: identified residue wrt insertion code: 'F'
I: identified residue wrt insertion code: 'F'
I: identified residue wrt insertion code: 'F'
I: identified residue wrt insertion code: 'F'
I: identified residue wrt insertion code: 'F'
I: identified residue wrt insertion code: 'F'
I: identified residue wrt insertion code: 'F'
I: identified residue wrt insertion code: 'F'
I: identified residue wrt insertion code: 'G'
I: identified residue wrt insertion code: 'G'
I: identified residue wrt insertion code: 'G'
I: identified residue wrt insertion code: 'G'
I: identified residue wrt insertio

## Step 4: Dock the ligand with Smina

In [ ]:
!./smina.static -r 1OYT-receptor.pdbqt -l 1OYT-FSN.pdbqt --autobox_ligand 1OYT-FSN.pdbqt --autobox_add 8 --exhaustiveness 16 -o 1OYT-redock.pdbqt


   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 1560346280

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------


In [ ]:
import py3Dmol

def drawit2(m,confId=-1):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p

def DrawComplex(protein,ligand):
    complex_pl = Chem.MolToPDBBlock(Chem.CombineMols(protein,ligand))
    #complex_mol=Chem.CombineMols(receptor,mols[-1])
    view = py3Dmol.view(width=600,height=600)
    view.addModel(complex_pl,'pdb')
    #view.addModel(Chem.MolToMolBlock(mols[0]),'sdf')
    chA = {'chain':['H','L','I']}
    chB = {'resn':'UNL'}
    view.setStyle(chA,{'cartoon': {'color':'spectrum'}})
    #view.setStyle(chA,{'lines': {}})
    view.addSurface(py3Dmol.VDW,{'opacity':0.8}, chB)
    view.setStyle(chB,{'stick':{}})
    view.zoomTo()
    return view   


In [ ]:
from rdkit import Chem

### Convert docked ligands from PDBQT to SDF

In [ ]:
!obabel 1OYT-redock.pdbqt -O 1OYT-redocked.sdf

9 molecules converted


In [ ]:
mols = [m for m in Chem.SDMolSupplier('1OYT-redocked.sdf') if m is not None]

In [ ]:
drawit2(mols[0])

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Step 5: Visualize the docked Protein-Ligand Complex

In [ ]:
receptor = Chem.MolFromPDBFile('1OYT-receptor.pdb')
DrawComplex(receptor,mols[0])

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
'''
complex_mol=Chem.CombineMols(receptor,mols[-1])
view = py3Dmol.view(width=600,height=600)
view.addModel(Chem.MolToPDBBlock(complex_mol),'pdb')
#view.addModel(Chem.MolToMolBlock(mols[0]),'sdf')
chA = {'chain':['H','L','I']}
chB = {'resn':'UNL'}
view.setStyle(chA,{'cartoon': {'color':'spectrum'}})
#view.setStyle(chA,{'lines': {}})
view.addSurface(py3Dmol.VDW,{'opacity':0.8}, chB)
view.setStyle(chB,{'stick':{}})
view.show()
'''

"\ncomplex_mol=Chem.CombineMols(receptor,mols[-1])\nview = py3Dmol.view(width=600,height=600)\nview.addModel(Chem.MolToPDBBlock(complex_mol),'pdb')\n#view.addModel(Chem.MolToMolBlock(mols[0]),'sdf')\nchA = {'chain':['H','L','I']}\nchB = {'resn':'UNL'}\nview.setStyle(chA,{'cartoon': {'color':'spectrum'}})\n#view.setStyle(chA,{'lines': {}})\nview.addSurface(py3Dmol.VDW,{'opacity':0.8}, chB)\nview.setStyle(chB,{'stick':{}})\nview.show()\n"